In [130]:
# import the libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from numpy import arange, sin, pi, random

# Global hyper-parameters
sequence_length = 1940
random_data_dup = 5  # each sample randomly duplicated between 0 and 9 times, see dropin function
epochs = 1
batch_size = 50

In [26]:
#algorith1  isolatation forecast!!  长度为x
def append_data(dic,key,value):
    if(dic.has_key(key)):
        dic[key].append(value)
    else:
        dic[key] = [value]
    return dic

def getStrs(pre,num):
    result = []
    for i in range(num):
        result.append(pre+str(i))
    return result

def  SeriesStandard(series):
    """
    return a new series which the mean is 0 and variance is 1
    :param series: 
    :return: 
    """
    mean = np.mean(series)
    variance = np.var(series)
    series = (series-mean)/variance
    return series

# consider delay threshold and missing segments
def get_range_proba(predict, label, delay=7):

    splits = np.where(label[1:] != label[:-1])[0] + 1
    is_anomaly = label[0] == 1
    new_predict = np.array(predict)
    pos = 0

    for sp in splits:
        if is_anomaly:
            if 1 in predict[pos:min(pos + delay + 1,sp)]:
                new_predict[pos: sp] = 1
            else:
                new_predict[pos: sp] = 0
        is_anomaly = not is_anomaly
        pos = sp
    sp = len(label)

    if is_anomaly:  #anomaly in the end
        if 1 in predict[pos: min(pos + delay+1,sp)]:
            new_predict[pos: sp] = 1
        else:
            new_predict[pos: sp] = 0
    return new_predict

# set missing = 0
def reconstruct_label(timestamp, label):
    timestamp = np.asarray(timestamp, np.int64)
    timestamp_sorted = np.asarray(timestamp[np.argsort(timestamp)])
    interval = np.min(np.diff(timestamp_sorted))
    if interval == 0:
        print(timestamp_sorted)
    idx = (timestamp_sorted - timestamp_sorted[0]) // interval
    new_label = np.zeros(shape=((timestamp_sorted[-1] - timestamp_sorted[0]) // interval + 1,), dtype=np.int)
    new_label[idx] = label
    return new_label

def label_evaluation(truth_df, result_df, delay=7):
    data = {'result': False, 'data': "", 'message': ""}
    kpi_names = truth_df['KPI ID'].values
    kpi_names = np.unique(kpi_names)
    y_true_list = []
    y_pred_list = []
    for kpi_name in kpi_names:
        truth = truth_df[truth_df["KPI ID"] == kpi_name]
        y_true = reconstruct_label(truth["timestamp"], truth["label"])
        try:
            result = result_df[result_df["KPI ID"] == kpi_name]
            y_pred = reconstruct_label(result["timestamp"], result["predict"])
        except:
            data['message'] = "The file you submitted need contain 'predict','timestamp' and  \
                             'KPI ID' columns"
            return json.dumps(data)
        try:
            assert np.array_equal(len(y_true),len(y_pred)) == True
        except:
            data['message'] = "The length of your submitted file is wrong"
            return json.dumps(data)

        y_pred = get_range_proba(y_pred, y_true, delay)
        y_true_list.append(y_true)
        y_pred_list.append(y_pred)

    fscore = f1_score(np.concatenate(y_true_list), np.concatenate(y_pred_list))
    data['result'] = True
    data['data'] = fscore
    data['message'] = 'success'
    print(json.dumps(data))
    return json.dumps(data)

def z_norm(result):
    result_mean = result.mean()
    result_std = result.std()
    result -= result_mean
    result /= result_std
    return result, result_mean

def dropin(X, y):
    """ The name suggests the inverse of dropout, i.e. adding more samples. See Data Augmentation section at
    http://simaaron.github.io/Estimating-rainfall-from-weather-radar-readings-using-recurrent-neural-networks/
    :param X: Each row is a training sequence
    :param y: Tne target we train and will later predict
    :return: new augmented X, y
    """
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    X_hat = []
    y_hat = []
    for i in range(0, len(X)):
        for j in range(0, np.random.random_integers(0, random_data_dup)):
            X_hat.append(X[i, :])
            y_hat.append(y[i, :])
    return np.asarray(X_hat), np.asarray(y_hat)

In [129]:
def make_samples(samples):
    data = samples.value
    print("Length of Data", len(data))
    # train data
    print("Creating train data...")
    result = []
    for index in range(0, len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    result = np.array(result)  # shape (samples, sequence_length)
    #result, result_mean = z_norm(result)
    
    print("Mean of train data : ", result_mean)
    print("Train data shape  : ", result.shape)
    np.random.shuffle(result)  # shuffles in-place
    X_train = result[:, :-500]
    y_train = result[:, -500:]
    X_train,y_train = dropin(X_train,y_train)
    X_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))
    y_train = np.reshape(y_train, (y_train.shape[0],1,y_train.shape[1]))
    return X_train,y_train

def make_submit_samples(samples):
    data = samples.value
    print("Length of Data", len(data))
    # train data
    print("Creating train data...")
    result = []
    for index in range(0, len(data) - 1440):
        result.append(data[index: index + 1440])
    result = np.array(result)  # shape (samples, sequence_length)
    #result, result_mean = z_norm(result)
    print("Mean of train data : ", result_mean)
    print("Train data shape  : ", result.shape)
    np.random.shuffle(result)  # shuffles in-place
    X_train = result
    X_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))
    return X_train

In [107]:
samples_train=pd.read_csv("./samples/samples_train.csv")
samples_testt=pd.read_csv("./samples/samples_test.csv")
X_train,y_train = make_samples(samples_train)
X_testt,y_testt = make_samples(samples_testt)
X_train.shape,y_train.shape

Length of Data 131795
Creating train data...
Mean of train data :  1.93798535255
Train data shape  :  (129855, 1940)
X shape: (129855, 1440)
y shape: (129855, 500)


/Users/sheldon/anaconda2/envs/dnn/lib/python3.6/site-packages/ipykernel_launcher.py:114: DeprecationWarning: This function is deprecated. Please call randint(0, 5 + 1) instead


Length of Data 131795
Creating train data...
Mean of train data :  1.94956887931
Train data shape  :  (129855, 1940)
X shape: (129855, 1440)
y shape: (129855, 500)


((323684, 1, 1440), (323684, 1, 500))

In [114]:
def build_model():
    model = Sequential()
    layers = {'input': 1, 'hidden1': 64, 'hidden2': 256, 'hidden3': 100, 'output': 1}
    
    model.add(LSTM(
        #input_length=sequence_length - 500,
        #input_dim=1,
        input_shape = (1,1440),
        output_dim=layers['hidden1'],
        return_sequences=True))
    
    model.add(Dropout(0.2))
    model.add(LSTM(
        layers['hidden2'],
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
        layers['hidden3'],
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(Dense(500))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mean_squared_error", optimizer="adam")
    print("Compilation Time : ", time.time() - start)
    return model

In [115]:
def run_network(model=None, data=None):
    global_start_time = time.time()
    #X_train,y_train = make_samples(samples_train)
    #X_testt,y_testt = make_samples(samples_testt)
    print('\nData Loaded. Compiling...\n')
    if model is None:
        model = build_model()

    try:
        print("Training...")
        model.fit(
                X_train, y_train,
                batch_size=batch_size, nb_epoch=epochs, validation_split=0.05)
        print("Predicting...")
        predicted = model.predict(X_testt)
        print("Reshaping predicted")
        predicted = np.reshape(predicted, (predicted.size,))
    except KeyboardInterrupt:
        print("prediction exception")
        print('Training duration (s) : ', time.time() - global_start_time)
    return model, y_testt, predicted

In [116]:
model, y_testt, predicted = run_network()


Data Loaded. Compiling...



/Users/sheldon/anaconda2/envs/dnn/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(1, 1440), return_sequences=True, units=64)`
  # Remove the CWD from sys.path while we load stuff.


Compilation Time :  0.038697004318237305
Training...


/Users/sheldon/anaconda2/envs/dnn/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 307499 samples, validate on 16185 samples
Epoch 1/1
307499/307499 [==============================] - 880s 3ms/step - loss: 0.2948 - val_loss: 0.2862
Predicting...
Reshaping predicted


In [131]:
X_testt = make_submit_samples(samples_testt)

Length of Data 131795
Creating train data...
Mean of train data :  1.95095794525
Train data shape  :  (130355, 1440)


In [132]:
predicted = model.predict(X_testt)

In [151]:
result=[]
for i in range(predicted.shape[0]):
    result.append(predicted[i][0][0])

In [152]:
result

[0.72790122,
 0.73831695,
 -0.70883572,
 0.69042224,
 -1.3081664,
 0.74861532,
 0.7249881,
 0.74354476,
 0.74002767,
 0.71699739,
 -1.0165126,
 -1.3081664,
 0.74920779,
 -1.3081664,
 -1.3081664,
 0.74933207,
 -1.3081664,
 0.69285452,
 0.68212575,
 -0.58691758,
 0.75105768,
 0.7403909,
 -1.3081664,
 -1.3081664,
 0.72103584,
 0.73871171,
 -1.3081664,
 -1.3081664,
 0.54793012,
 0.74745578,
 -1.3081664,
 0.68049002,
 0.74066085,
 0.69346833,
 -0.26070744,
 -0.97967762,
 -1.3081664,
 0.7516886,
 -1.2217715,
 0.7437408,
 0.74999243,
 0.7495265,
 0.17807968,
 0.64529711,
 0.7365948,
 -0.85505122,
 -1.3081664,
 -0.81969988,
 0.69502145,
 -1.2529246,
 0.72292811,
 0.75267786,
 0.73616374,
 0.75402904,
 -1.1580369,
 -1.3081664,
 0.75069642,
 -0.55524391,
 0.74887496,
 -1.3081664,
 0.71445704,
 0.75068313,
 0.74928415,
 0.75343615,
 -0.81672645,
 0.39331874,
 0.65485656,
 0.73888427,
 -1.3081664,
 0.74666029,
 0.74911827,
 0.75204659,
 -1.3081664,
 0.74528778,
 -0.9094196,
 0.74633455,
 0.8057800

In [142]:
perdict=[]

for i in range(predicted.shape

array([[[ 0.72790122,  0.69380295,  0.72190005, ...,  0.679748  ,
          0.66979218,  0.66963685]],

       [[ 0.73831695,  0.70347065,  0.73186731, ...,  0.68968385,
          0.68024284,  0.68007743]],

       [[-0.70883572, -0.69215208, -0.68276948, ..., -0.68147469,
         -0.67411649, -0.68736702]],

       ..., 
       [[ 0.73265785,  0.69806486,  0.72649056, ...,  0.68436778,
          0.67472762,  0.67434263]],

       [[-1.30816638, -1.26406991, -1.26698267, ..., -1.22115231,
         -1.21254158, -1.21884441]],

       [[-0.57927239, -0.56587887, -0.55408984, ..., -0.5532884 ,
         -0.55070627, -0.56250972]]], dtype=float32)

In [3]:
filepath1="../../dataset/anomaly_detection_1/train.csv"
data=pd.read_csv(filepath1)
filepath2 = "../../dataset/anomaly_detection_1/test_dataset.csv"
submit = pd.read_csv(filepath2)
KPI_ID = pd.unique(data['KPI ID'])
KPI_ID00 = data[data['KPI ID']==KPI_ID[0]]
KPI_ID01 = submit[submit['KPI ID']==KPI_ID[1]]
samples = make_samples(KPI_ID[0],np.array(KPI_ID00['timestamp']),np.array(KPI_ID00['value']),np.array(KPI_ID00['label']),40)
for kpi_id in KPI_ID[1:27]:
    kpi_data = data[data['KPI ID']==kpi_id]
    tmp = make_samples(kpi_id,np.array(kpi_data['timestamp']),np.array(kpi_data['value']),np.array(kpi_data['label']),40)
    samples = pd.concat([samples,tmp])
KPI_ID01 = submit[submit['KPI ID']==KPI_ID[0]]
submit_samples = make_samples_without_label(KPI_ID[0],np.array(KPI_ID00['timestamp']),np.array(KPI_ID00['value']),40)
for kpi_id in KPI_ID[1:27]:
    kpi_data = submit[submit['KPI ID']==kpi_id]
    tmp = make_samples_without_label(kpi_id,np.array(kpi_data['timestamp']),np.array(kpi_data['value']),40)
    submit_samples = pd.concat([submit_samples,tmp])
samples_train = samples[samples.label==0]

In [7]:
#Isolation Forest
from sklearn.ensemble import IsolationForest

clf = IsolationForest(n_estimators=30,max_samples = 0.0005,verbose=2)
clf.fit(samples_train.iloc[:,0:41])
y_pred_sampes=clf.predict(samples.iloc[:,0:41])
y_pred_submit=clf.predict(submit_samples.iloc[:,0:41])
samples['predict']=y_pred_sampes
submit_samples['predict']=y_pred_submit
data   = pd.merge(data,samples[["id","time","predict"]],left_on=["KPI ID","timestamp"],right_on=["id","time"],how='left')
submit=pd.merge(submit,submit_samples[["id","time","predict"]],left_on=["KPI ID","timestamp"],right_on=["id","time"],how='left')
data = data[["KPI ID","timestamp","label","predict"]]
data["predict"][np.isnan(data.predict)]=0
data["predict"][data.predict==1]=0
data["predict"][data.predict==-1]=1

Building estimator 1 of 30 for this parallel run (total 30)...
Building estimator 2 of 30 for this parallel run (total 30)...
Building estimator 3 of 30 for this parallel run (total 30)...
Building estimator 4 of 30 for this parallel run (total 30)...
Building estimator 5 of 30 for this parallel run (total 30)...
Building estimator 6 of 30 for this parallel run (total 30)...
Building estimator 7 of 30 for this parallel run (total 30)...
Building estimator 8 of 30 for this parallel run (total 30)...
Building estimator 9 of 30 for this parallel run (total 30)...
Building estimator 10 of 30 for this parallel run (total 30)...
Building estimator 11 of 30 for this parallel run (total 30)...
Building estimator 12 of 30 for this parallel run (total 30)...
Building estimator 13 of 30 for this parallel run (total 30)...
Building estimator 14 of 30 for this parallel run (total 30)...
Building estimator 15 of 30 for this parallel run (total 30)...
Building estimator 16 of 30 for this parallel run

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   49.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   49.2s finished
/Users/sheldon/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sheldon/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [9]:
submit["predict"][np.isnan(submit.predict)]=0
submit["predict"][submit.predict==1]=0
submit["predict"][submit.predict==-1]=1
submit.head()

/Users/sheldon/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sheldon/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sheldon/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imp

,KPI ID,timestamp,value,id,time,predict
0,02e99bd4f6cfb33f,1501475700,2.719298,NaN,NaN,0.0
1,02e99bd4f6cfb33f,1501475760,2.456140,NaN,NaN,0.0
2,02e99bd4f6cfb33f,1501475820,2.333333,NaN,NaN,0.0
3,02e99bd4f6cfb33f,1501475880,2.543860,NaN,NaN,0.0
4,02e99bd4f6cfb33f,1501475940,2.333333,NaN,NaN,0.0


In [15]:
submit['predict'] = submit['predict'].astype(np.int)
submit[["KPI ID","timestamp","predict"]].to_csv("submit1.csv",index=False)

In [16]:
submit.head()

,KPI ID,timestamp,value,id,time,predict
0,02e99bd4f6cfb33f,1501475700,2.719298,NaN,NaN,0
1,02e99bd4f6cfb33f,1501475760,2.456140,NaN,NaN,0
2,02e99bd4f6cfb33f,1501475820,2.333333,NaN,NaN,0
3,02e99bd4f6cfb33f,1501475880,2.543860,NaN,NaN,0
4,02e99bd4f6cfb33f,1501475940,2.333333,NaN,NaN,0


In [13]:
label_evaluation(data.iloc[:,[0,1,2]],data.iloc[:,[0,1,3]])

{"message": "success", "data": 0.03246555002012184, "result": true}


'{"message": "success", "data": 0.03246555002012184, "result": true}'

In [29]:
data2["predict"][np.isnan(data2.predict)]=0
data2["predict"][data2.predict==1]=0
data2["predict"][data2.predict==-1]=1
data2.head(50)

/Users/sheldon/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sheldon/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,KPI ID,timestamp,label,predict
0,02e99bd4f6cfb33f,1493568000,0,0.0
1,02e99bd4f6cfb33f,1493568060,0,0.0
2,02e99bd4f6cfb33f,1493568120,0,0.0
3,02e99bd4f6cfb33f,1493568180,0,0.0
4,02e99bd4f6cfb33f,1493568240,0,0.0
5,02e99bd4f6cfb33f,1493568300,0,0.0
6,02e99bd4f6cfb33f,1493568360,0,0.0
7,02e99bd4f6cfb33f,1493568420,0,0.0
8,02e99bd4f6cfb33f,1493568480,0,0.0
9,02e99bd4f6cfb33f,1493568540,0,0.0


In [128]:
samples_train = samples[samples.label==0]
[len(samples),len(samples_train)]

[131755, 129624]

In [129]:
#Isolation Forest
from sklearn.ensemble import IsolationForest

clf = IsolationForest(n_estimators=500,max_samples = 0.005)
clf.fit(samples_train.iloc[:,0:40])
y_pred_sampes=clf.predict(samples.iloc[:,0:40])
y_pred_sampes

array([1, 1, 1, ..., 1, 1, 1])

In [14]:
len(data)

2617679

In [151]:
arr=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
arr=arr.extend(y_pred_sampes)
arr

In [168]:
KPI_ID00 = data[data['KPI ID']==KPI_ID[0]]

In [169]:
arr=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
arr.extend(y_pred_sampes)
KPI_ID00["predict"]=arr

/Users/sheldon/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [170]:
KPI_ID00["predict"][KPI_ID00.predict==1]=0
KPI_ID00["predict"][KPI_ID00.predict==-1]=1
KPI_ID00.head()
KPI_ID00.iloc[:,[0,1,4]].to_csv("predict.csv")

/Users/sheldon/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sheldon/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [80]:
result=make_samples([1,2,3,4,5,6,7,8],[1,1,1,0,1,0,1,1],3)

In [81]:
result

,before0,before1,before2,before3,label
0,4,3,2,1,0
1,5,4,3,2,1
2,6,5,4,3,0
3,7,6,5,4,1
4,8,7,6,5,1


str